<a href="https://colab.research.google.com/github/harshit0413/Darkweb-Analyzer-/blob/main/Darkweb-Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1st Level Classification**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.4 MB/s eta 0:00:00


In [3]:
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444328 sha256=9685ca89573958596eee2e4300fed598889a18c1a335517870dd2d2aae555465
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Load the labeled dataset
df_labeled = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/darknet-normal.csv')

df_labeled.replace([np.inf, -np.inf], np.nan, inplace=True)

df_labeled.fillna(0, inplace=True)  # or

# Define the features to keep, based on the extract_features function
features_to_keep = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Packet Length Min', 'Packet Length Mean', 'Fwd IAT Total',
    'Flow IAT Min', 'Flow IAT Max', 'Fwd IAT Mean', 'Flow Packets/s',
    'Flow Bytes/s', 'Idle Min', 'Idle Max', 'Idle Mean',
    'Idle Std', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'ACK Flag Count'
]

# Keep only the relevant features
df_relevant_features = df_labeled[features_to_keep + ['Label']]

# Split the data into features and labels
X = df_relevant_features.drop('Label', axis=1)

y = df_relevant_features['Label'].map({'Normal': 'normal', 'FreeNet': 'darknet', 'I2P': 'darknet', 'Tor': 'darknet', 'ZeroNet': 'darknet'})

df_labeled.fillna(0, inplace=True)

# Split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply one-hot encoding only to the train dataset to avoid memory issues
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Align X_train and X_test to ensure they have the same columns
X_train, X_test = X_train.align(X_test, join='inner', axis=1)

# Initialize XGBoost classifier
Xgb_classify = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=42)

# Encode the labels with LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Train the classifier with the encoded binary labels
Xgb_classify.fit(X_train,y_train_encoded)

# Make predictions with the encoded labels
encoded_predictions = Xgb_classify.predict(X_test)

# Decode the predictions back to original labels
predictions = label_encoder.inverse_transform(encoded_predictions)
accuracy = accuracy_score(y_test_encoded, encoded_predictions)
precision = precision_score(y_test_encoded, encoded_predictions, pos_label=label_encoder.transform(['darknet'])[0])
recall = recall_score(y_test_encoded, encoded_predictions, pos_label=label_encoder.transform(['darknet'])[0])
f1 = f1_score(y_test_encoded, encoded_predictions, pos_label=label_encoder.transform(['darknet'])[0])


print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Accuracy: 0.9944990484135882
Precision: 0.9959913326110509
Recall: 0.9926037898828484
F1 Score: 0.9942946759321851


In [5]:
#saving the model
Xgb_classify.save_model('xgb_model.json')

In [6]:
#load the model
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('xgb_model.json')

In [7]:
from scapy.all import rdpcap, IP, TCP
import numpy as np
import pandas as pd


def extract_features(pcap_file):
    packets = rdpcap(pcap_file)
    features = {
        'Flow Duration': 0.0,
        'Total Fwd Packet': 0,
        'Total Bwd packets': 0,
        'Packet Length Min': np.inf,
        'Packet Length Mean': 0.0,
        'Fwd IAT Total': 0.0,
        'Flow IAT Min': np.inf,
        'Flow IAT Max': 0.0,
        'Fwd IAT Mean': 0.0,
        'Flow Packets/s': 0.0,
        'Flow Bytes/s': 0.0,
        'Idle Min': np.inf,
        'Idle Max': 0.0,
        'Idle Mean': 0.0,
        'Idle Std': 0.0,
        'FWD Init Win Bytes': 0,
        'Bwd Init Win Bytes': 0,
        'ACK Flag Count': 0
    }

    if not packets:
        return pd.DataFrame(features, index=[0])

    start_times = []
    packet_lengths = []
    iats = []
    total_bytes = 0

    for packet in packets:
        if IP in packet and TCP in packet:
            packet_length = len(packet)
            packet_lengths.append(packet_length)
            total_bytes += packet_length

            if 'S' in packet[TCP].flags:
                if features['FWD Init Win Bytes'] == 0:
                    features['FWD Init Win Bytes'] = packet[TCP].window
                else:
                    features['Bwd Init Win Bytes'] = packet[TCP].window

            if 'A' in packet[TCP].flags:
                features['ACK Flag Count'] += 1

            start_times.append(float(packet.time))

            if len(start_times) > 1:
                iat = start_times[-1] - start_times[-2]
                iats.append(iat)

    features['Flow Duration'] = max(start_times) - min(start_times)
    features['Total Fwd Packet'] = len([p for p in packets if IP in p and p[IP].src < p[IP].dst])
    features['Total Bwd packets'] = len([p for p in packets if IP in p and p[IP].src > p[IP].dst])
    features['Packet Length Min'] = min(packet_lengths)
    features['Packet Length Mean'] = np.mean(packet_lengths) if packet_lengths else 0
    features['Fwd IAT Total'] = sum(iats)
    features['Flow IAT Min'] = min(iats) if iats else 0
    features['Flow IAT Max'] = max(iats) if iats else 0
    features['Flow IAT Min'] = np.mean(iats) if iats else 0
    features['Flow Packets/s'] = len(packets) / features['Flow Duration'] if features['Flow Duration'] else 0
    features['Flow Bytes/s'] = total_bytes / features['Flow Duration'] if features['Flow Duration'] else 0

    # Handle potential NaNs and infs
    for key, value in features.items():
        if isinstance(value, float) and (np.isinf(value) or np.isnan(value)):
            features[key] = 0

    df_features = pd.DataFrame([features])

# Handle potential NaNs and infs again before returning
    df_features.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_features.fillna(0, inplace=True)

    return df_features


In [8]:
import xgboost as xgb

def classify_traffic(df_features, model_path):
    # Load the trained model
    xgb_model = xgb.XGBClassifier()

    # Load the model
    xgb_model.load_model(model_path)

    # Ensure that the model has been fitted before making predictions
    if not xgb_model.get_booster().attr("n_features"):
        raise ValueError("Model needs to be fitted before making predictions")

    # Predict the traffic class
    predictions = xgb_model.predict(df_features)
    return predictions


In [9]:
import xgboost as xgb

model_path = '/content/xgb_model.json'  # Make sure this is the correct path to your model file
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/zeronet-p2p_00001_20200421125502.pcap'

# Loading the trained XGBoost model
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.load_model(model_path)

# Extract features from the pcap file
df_features = extract_features(pcap_file_path)

# Output the predicted class
print("Predicted Class:", predictions[0])
print(label_encoder.classes_)

Predicted Class: darknet
['darknet' 'normal']


In [10]:
if isinstance(df_features, pd.Series):
    df_features = df_features.to_frame().transpose()

predictions = xgb_classifier.predict(df_features)

In [11]:
if predictions[0] == 0:
    print("Darknet")
else:
    print("Normal")

Darknet


# **2nd Level Classification**

In [12]:
!pip install scapy

In [13]:
from sklearn.preprocessing import StandardScaler

# Assuming numerical features are already identified in features_to_keep
numerical_features = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Flow Bytes/s', 'Flow Packets/s', 'Fwd IAT Mean',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
    'Packet Length Min', 'Packet Length Mean',
    'ACK Flag Count', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
    'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
X_train_scaled = scaler.fit_transform(X_train[numerical_features])
X_test_scaled = scaler.transform(X_test[numerical_features])

# Save the scaler for future use
import joblib
joblib.dump(scaler, 'scaler.save')
from sklearn.preprocessing import StandardScaler

# Assuming numerical features are already identified in features_to_keep
numerical_features = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Flow Bytes/s', 'Flow Packets/s', 'Fwd IAT Mean',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
    'Packet Length Min', 'Packet Length Mean',
    'ACK Flag Count', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
    'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
X_train_scaled = scaler.fit_transform(X_train[numerical_features])
X_test_scaled = scaler.transform(X_test[numerical_features])

# Save the scaler for future use
import joblib
joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import joblib

# Load dataset
df_labeled = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/darknet-type.csv')

# Convert non-numeric columns to numeric
label_encoders = {}
for column in df_labeled.columns:
    if df_labeled[column].dtype == 'object' and column != 'Label':
        le = LabelEncoder()
        df_labeled[column] = le.fit_transform(df_labeled[column].astype(str))
        label_encoders[column] = le

# Replace inf/-inf with NaN and impute missing values
df_labeled.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='median')
df_labeled.iloc[:, df_labeled.columns != 'Label'] = imputer.fit_transform(df_labeled.iloc[:, df_labeled.columns != 'Label'])

# Define and extract features
features_to_keep = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Flow Bytes/s', 'Flow Packets/s', 'Fwd IAT Mean',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
    'Packet Length Min', 'Packet Length Mean', 'ACK Flag Count',
    'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'Idle Mean',
    'Idle Std', 'Idle Max', 'Idle Min',

 ]  # List of features as provided
X = df_labeled[features_to_keep]
y = df_labeled['Label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the hyperparameter space for Random Search
param_dist = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 400, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'reg_lambda': [0.01, 0.1, 1, 10],
    'reg_alpha': [0, 0.1, 0.5, 1]
}

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', use_label_encoder=False, num_class=len(np.unique(y_train_encoded)), seed=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings that are sampled
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV to the scaled training data
random_search.fit(X_train_scaled, y_train_encoded)

# Select the best model
best_model = random_search.best_estimator_

# Prediction using the best model
predictions_encoded = best_model.predict(X_test_scaled)
predictions = label_encoder.inverse_transform(predictions_encoded)

# Performance metrics
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test_encoded, predictions_encoded, average='weighted')

print(f"Enhanced Accuracy: {accuracy}")
print(f"Enhanced F1 Score: {f1}")

# Save the scaler and the best model
joblib.dump(scaler, 'scaler.save')
best_model.save_model('xgboost_model.json')

<ipython-input-17-718d1c5f3f5a>:24: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_labeled.iloc[:, df_labeled.columns != 'Label'] = imputer.fit_transform(df_labeled.iloc[:, df_labeled.columns != 'Label'])


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Enhanced Accuracy: 0.9636363636363636
Enhanced F1 Score: 0.9636151738117704


In [18]:
import joblib

In [19]:
joblib.dump(label_encoder, 'label_encoder.save')

['label_encoder.save']

In [20]:
from scapy.all import rdpcap, IP, TCP
import numpy as np
import pandas as pd


def extract_features(pcap_file):
    packets = rdpcap(pcap_file)
    features = {
        'Flow Duration': 0.0,
        'Total Fwd Packet': 0,
        'Total Bwd packets': 0,
        'Flow Bytes/s': 0.0,
        'Flow Packets/s': 0.0,
        'Fwd IAT Mean': 0.0,
        'Flow IAT Max': 0.0,
        'Flow IAT Min': np.inf,
        'Fwd IAT Total': 0.0,
        'Packet Length Min': np.inf,
        'Packet Length Mean': 0.0,
        'ACK Flag Count': 0,
        'FWD Init Win Bytes': 0,
        'Bwd Init Win Bytes': 0,
        'Idle Mean': 0.0,
        'Idle Std': 0.0,
        'Idle Max': 0.0,
        'Idle Min': np.inf,

    }
    scaler = joblib.load('scaler.save')

    if not packets:
        return pd.DataFrame(features, index=[0])

    start_times = []
    packet_lengths = []
    iats = []
    total_bytes = 0

    for packet in packets:
        if IP in packet and TCP in packet:
            packet_length = len(packet)
            packet_lengths.append(packet_length)
            total_bytes += packet_length

            if 'S' in packet[TCP].flags:
                if features['FWD Init Win Bytes'] == 0:
                    features['FWD Init Win Bytes'] = packet[TCP].window
                else:
                    features['Bwd Init Win Bytes'] = packet[TCP].window

            if 'A' in packet[TCP].flags:
                features['ACK Flag Count'] += 1

            start_times.append(float(packet.time))

            if len(start_times) > 1:
                iat = start_times[-1] - start_times[-2]
                iats.append(iat)

    features['Flow Duration'] = max(start_times) - min(start_times)
    features['Total Fwd Packet'] = len([p for p in packets if IP in p and p[IP].src < p[IP].dst])
    features['Total Bwd packets'] = len([p for p in packets if IP in p and p[IP].src > p[IP].dst])
    features['Packet Length Min'] = min(packet_lengths)
    features['Packet Length Mean'] = np.mean(packet_lengths) if packet_lengths else 0
    features['Fwd IAT Total'] = sum(iats)
    features['Flow IAT Min'] = min(iats) if iats else 0
    features['Flow IAT Max'] = max(iats) if iats else 0
    features['Flow IAT Min'] = np.mean(iats) if iats else 0
    features['Flow Packets/s'] = len(packets) / features['Flow Duration'] if features['Flow Duration'] else 0
    features['Flow Bytes/s'] = total_bytes / features['Flow Duration'] if features['Flow Duration'] else 0

    # Handle potential NaNs and infs
    for key, value in features.items():
        if isinstance(value, float) and (np.isinf(value) or np.isnan(value)):
            features[key] = 0

    df_features = pd.DataFrame([features])


# Handle potential NaNs and infs again before returning
    df_features.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_features.fillna(0, inplace=True)
    df_features_scaled = scaler.transform(df_features[numerical_features])

    return pd.DataFrame(df_features_scaled, columns=numerical_features)

In [25]:
import xgboost as xgb

# Define the path to your model and pcap file
model_path = '/content/xgboost_model.json'  # Make sure this is the correct path to your model file
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/zeronet_00001_20200421125502.pcap'

# Load the trained XGBoost model
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.load_model(model_path)

# Extract features from the pcap file
df_features = extract_features(pcap_file_path)

# Prepare the features for the model (make sure df_features is a DataFrame with the correct shape)
# The model expects a 2D array-like structure. If df_features is a Series, convert it to a DataFrame

# Check the predicted class and print the corresponding label


# Output the predicted class
#print("Predicted Class:", predictions[0])

In [26]:
if isinstance(df_features, pd.Series):
    df_features = df_features.to_frame().transpose()

# Predict using the loaded model
# Ensure df_features does not have an index column or any non-numeric columns that the model wasn't trained on
predictions = xgb_classifier.predict(df_features)

In [27]:
# Save the model to a binary file
best_model.save_model('xgblevel2_model.json')

In [28]:
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('xgblevel2_model.json')

In [ ]:
print(label_encoder.classes_)  # This will show you the mapping
print(predictions[0])

In [31]:
import xgboost as xgb
import joblib
import pandas as pd

# Define paths
model_path = '/content/xgboost_model.json'
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/zeronet_00001_20200506213643.pcap'
scaler_path = '/content/scaler.save'
label_encoder_path = '/content/label_encoder.save'

# Load the trained XGBoost model, scaler, and LabelEncoder
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.load_model(model_path)
scaler = joblib.load(scaler_path)
label_encoder = joblib.load(label_encoder_path)

# Extract features from the pcap file
df_features = extract_features(pcap_file_path)

# Scale the extracted features
df_features_scaled = scaler.transform(df_features)

# Predict using the scaled features
predictions_encodeds = xgb_classifier.predict(df_features_scaled)

# Decode the predictions to original class labels
predict = label_encoder.inverse_transform(predictions_encodeds)

# Output the prediction
print(label_encoder.classes_)  # This shows the mapping
print(predict[0])  # This shows the predicted class

['FreeNet' 'I2P' 'Tor' 'ZeroNet']
ZeroNet


**3 RD LEVEL CLASSIFIER-ZERONET**

In [32]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

# Load data from CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/zeronet-behavior.csv')

# Drop rows with missing values
data.dropna(inplace=True)

# Replace infinite values with NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with zeros
data.fillna(0, inplace=True)

# Define the features to keep
features_to_keep = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Packet Length Min', 'Packet Length Mean', 'Fwd IAT Total',
    'Flow IAT Min', 'Flow IAT Max', 'Fwd IAT Mean', 'Flow Packets/s',
    'Flow Bytes/s', 'Idle Min', 'Idle Max', 'Idle Mean',
    'Idle Std', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'ACK Flag Count'
]

# Keep only the relevant features
df_relevant_features = data[features_to_keep + ['Label']]

# Convert label column to numerical values using LabelEncoder
label_encoder = LabelEncoder()
df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])

# Split the data into features and labels
X = df_relevant_features.drop('Label', axis=1)
y = df_relevant_features['Label']

# One-hot encode categorical columns
categorical_columns = X.select_dtypes(include='object').columns
X_encoded = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Creating LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Setting parameters for LightGBM
params = {
    'objective': 'multiclass',
    'num_class': len(data['Label'].unique()),
    'metric': 'multi_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Training the model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Making predictions
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to predicted classes

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("F1 Score:", f1)


<ipython-input-32-00239b23561b>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Accuracy: 0.9205426356589147
F1 Score: 0.9204826210689786


In [33]:
bst.save_model('trained_lightgbm_model.txt')

In [34]:
loaded_model = lgb.Booster(model_file='trained_lightgbm_model.txt')


In [35]:
import lightgbm as lgb

def classify_trafficlgb(df_features, model_path):
    # Load the trained model
    lgb_model = lgb.Booster(model_file=model_path)

    # Predict the traffic class
    predictions = lgb_model.predict(df_features)

    # Convert probability predictions to class labels
    predicted_classes = predictions.argmax(axis=1)

    return predicted_classes


In [36]:
import lightgbm as lgb

# Define the path to your model and pcap file
model_path = '/content/trained_lightgbm_model.txt'  # Make sure this is the correct path to your model file
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/zeronet_00001_20200421125502.pcap'

# Load the trained LightGBM model
lgb_model = lgb.Booster(model_file=model_path)

# Extract features from the pcap file
df_features = extract_features(pcap_file_path)

# Prepare the features for the model (make sure df_features is a DataFrame with the correct shape)
# The model expects a 2D array-like structure. If df_features is a Series, convert it to a DataFrame

# Check the predicted class and print the corresponding label
predictionz = lgb_model.predict(df_features)
predicted_class = label_encoder.inverse_transform(predictionz.argmax(axis=1))[0]
print("Predicted Class:", predicted_class)


Predicted Class: browsing


**TOR**

In [38]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data from CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/tor-behavior.csv')

# Drop rows with missing values
data.dropna(inplace=True)

# Identify and encode categorical variables
categorical_columns = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

# Separate features and target variable




data.dropna(inplace=True)

# Replace infinite values with NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with zeros
data.fillna(0, inplace=True)

# Define the features to keep
features_to_keep = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Packet Length Min', 'Packet Length Mean', 'Fwd IAT Total',
    'Flow IAT Min', 'Flow IAT Max', 'Fwd IAT Mean', 'Flow Packets/s',
    'Flow Bytes/s', 'Idle Min', 'Idle Max', 'Idle Mean',
    'Idle Std', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'ACK Flag Count'
]

# Keep only the relevant features
df_relevant_features = data[features_to_keep + ['Label']]

X = df_relevant_features.drop('Label', axis=1)
y = df_relevant_features['Label']

# Convert label column to numerical values using LabelEncoder
label_encoder = LabelEncoder()
df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])


# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

# Convert data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Setting parameters for XGBoost
params = {
    'objective': 'multi:softmax',  # Assuming you have multiple classes
    'num_class': len(data['Label'].unique()),  # Number of unique classes
    'eval_metric': 'mlogloss',  # you can use different metrics here
    'max_depth': 6,
    'eta': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.9,
    'verbosity': 0
}

# Training the model
num_round = 100
bst = xgb.train(params, dtrain, num_round, evals=[(dtest, 'eval')])

# Making predictions
y_predt = bst.predict(dtest)


<ipython-input-38-d2b3611ec250>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])


[0]	eval-mlogloss:1.97284
[1]	eval-mlogloss:1.88553
[2]	eval-mlogloss:1.80949
[3]	eval-mlogloss:1.74308
[4]	eval-mlogloss:1.68023
[5]	eval-mlogloss:1.62412
[6]	eval-mlogloss:1.57255
[7]	eval-mlogloss:1.52437
[8]	eval-mlogloss:1.48040
[9]	eval-mlogloss:1.43919
[10]	eval-mlogloss:1.40200
[11]	eval-mlogloss:1.36810
[12]	eval-mlogloss:1.33607
[13]	eval-mlogloss:1.30549
[14]	eval-mlogloss:1.27656
[15]	eval-mlogloss:1.24989
[16]	eval-mlogloss:1.22560
[17]	eval-mlogloss:1.20259
[18]	eval-mlogloss:1.17969
[19]	eval-mlogloss:1.15965
[20]	eval-mlogloss:1.13993
[21]	eval-mlogloss:1.12096
[22]	eval-mlogloss:1.10286
[23]	eval-mlogloss:1.08573
[24]	eval-mlogloss:1.06981
[25]	eval-mlogloss:1.05431
[26]	eval-mlogloss:1.03964
[27]	eval-mlogloss:1.02540
[28]	eval-mlogloss:1.01192
[29]	eval-mlogloss:0.99877
[30]	eval-mlogloss:0.98691
[31]	eval-mlogloss:0.97560
[32]	eval-mlogloss:0.96429
[33]	eval-mlogloss:0.95413
[34]	eval-mlogloss:0.94365
[35]	eval-mlogloss:0.93419
[36]	eval-mlogloss:0.92444
[37]	eval-m

In [39]:
# Save the model to a binary file
bst.save_model('xgblevel3t_model.json')



In [40]:
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('xgblevel3t_model.json')

In [41]:
import xgboost as xgb

def classify_traffict(df_features, model_path):
    # Load the trained model
    xgb_model = xgb.XGBClassifier()

    # Load the model
    xgb_model.load_model(model_path)

    # Ensure that the model has been fitted before making predictions
    if not xgb_model.get_booster().attr("n_features"):
        raise ValueError("Model needs to be fitted before making predictions")

    # Predict the traffic class
    predictions = xgb_model.predict(df_features)
    return predictions

In [47]:
import xgboost as xgb

# Define the path to your model and pcap file
model_path = '/content/xgblevel3t_model.json'  # Make sure this is the correct path to your model file
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/tor_00001_20200417155424.pcap'

# Load the trained XGBoost model
bst = xgb.XGBClassifier()
bst.load_model(model_path)

# Extract features from the pcap file
df_features = extract_features(pcap_file_path)

# Prepare the features for the model (make sure df_features is a DataFrame with the correct shape)
# The model expects a 2D array-like structure. If df_features is a Series, convert it to a DataFrame

# Check the predicted class and print the corresponding label


# Output the predicted class
#print("Predicted Class:", predictions[0])


In [67]:
if isinstance(df_features, pd.Series):
    df_features = df_features.to_frame().transpose()

# Predict using the loaded model
# Ensure df_features does not have an index column or any non-numeric columns that the model wasn't trained on
predictiont = bst.predict(df_features)
predicted_class = label_encoder.inverse_transform(predictionz.argmax(axis=1))[0]
print("Predicted Class:", predicted_class)



Predicted Class: chat


  FREENET

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import lightgbm as lgb

# Load data from CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/freenet-behavior.csv')

# Drop rows with missing values
data.dropna(inplace=True)

# Replace infinite values with NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with zeros
data.fillna(0, inplace=True)

# Define the features to keep
features_to_keep = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Packet Length Min', 'Packet Length Mean', 'Fwd IAT Total',
    'Flow IAT Min', 'Flow IAT Max', 'Fwd IAT Mean', 'Flow Packets/s',
    'Flow Bytes/s', 'Idle Min', 'Idle Max', 'Idle Mean',
    'Idle Std', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'ACK Flag Count'
]

# Keep only the relevant features
df_relevant_features = data[features_to_keep + ['Label']]

# Convert label column to numerical values using LabelEncoder
label_encoder = LabelEncoder()
df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])

# Split the data into features and labels
X = df_relevant_features.drop('Label', axis=1)
y = df_relevant_features['Label']

# One-hot encode categorical columns
categorical_columns = X.select_dtypes(include='object').columns
X_encoded = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Setting parameters for LightGBM
params = {
    'objective': 'multiclass',
    'num_class': len(df_relevant_features['Label'].unique()),
    'metric': 'multi_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Training the model
train_data = lgb.Dataset(X_train, label=y_train)
bst = lgb.train(params, train_data)

# Making predictions
y_pred = bst.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to predicted classes

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("F1 Score:", f1)


<ipython-input-51-ef16c09078ee>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.963392312385601
F1 Score: 0.9634453142252268


In [52]:
import lightgbm as lgb

# Assume 'bst' is your trained LightGBM model

# Save the model
bst.save_model('gbdt_model.txt')


In [53]:
# Load the model
loaded_model = lgb.Booster(model_file='gbdt_model.txt')


In [54]:
import numpy as np

def classify_traffic_gbdt(df_features, model_path):
    """
    Classifies traffic using a GBDT model.

    Parameters:
    - df_features (2D array-like): Features of the traffic data.
    - model_path (str): Path to the trained GBDT model file.

    Returns:
    - predicted_classes (array-like): Predicted classes for the traffic data.
    """
    # Load the trained GBDT model
    model = loaded_model(model_path)

    # Predict the traffic class probabilities
    predictions = model.predict(df_features)

    # Convert probability predictions to class labels
    predicted_classes = np.argmax(predictions, axis=1)

    return predicted_classes


In [55]:
import lightgbm as lgb

# Define the path to your model and pcap file
model_path = '/content/gbdt_model.txt'  # Make sure this is the correct path to your model file
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/freenet-1_00001_20200416195121.pcap'

# Load the trained LightGBM model
lgb_model = lgb.Booster(model_file=model_path)

# Extract features from the pcap file (make sure you have a function named 'extract_features' for this)
df_features = extract_features(pcap_file_path)

# Prepare the features for the model (make sure df_features is a DataFrame with the correct shape)
# The model expects a 2D array-like structure. If df_features is a Series, convert it to a DataFrame

# Check the predicted class and print the corresponding label
predictions = lgb_model.predict(df_features)
predicted_class = predictions.argmax(axis=1)[0]  # Assuming you're predicting only one instance
print("Predicted Class:", predicted_class)


Predicted Class: 3


In [56]:
predicted_class = label_encoder.inverse_transform(predictions.argmax(axis=1))[0]

In [57]:
print("Predicted Class:", predicted_class)


Predicted Class: ftp


I2P

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import xgboost as xgb

# Load data from CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/i2p-behavior.csv')

# Drop rows with missing values
data.dropna(inplace=True)

# Replace infinite values with NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with zeros
data.fillna(0, inplace=True)

# Define the features to keep
features_to_keep = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Packet Length Min', 'Packet Length Mean', 'Fwd IAT Total',
    'Flow IAT Min', 'Flow IAT Max', 'Fwd IAT Mean', 'Flow Packets/s',
    'Flow Bytes/s', 'Idle Min', 'Idle Max', 'Idle Mean',
    'Idle Std', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'ACK Flag Count'
]

# Keep only the relevant features
df_relevant_features = data[features_to_keep + ['Label']]

# Convert label column to numerical values using LabelEncoder
label_encoder = LabelEncoder()
df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])

# Split the data into features and labels
X = df_relevant_features.drop('Label', axis=1)
y = df_relevant_features['Label']

# One-hot encode categorical columns
categorical_columns = X.select_dtypes(include='object').columns
X_encoded = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Define parameters for XGBoost
params = {
    'objective': 'multi:softmax',
    'num_class': len(df_relevant_features['Label'].unique()),
    'eval_metric': 'mlogloss',
    'max_depth': 6,
    'eta': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.9,
    'verbosity': 0
}

# Convert data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Training the model
num_round = 100
bstI2P = xgb.train(params, dtrain, num_round)

# Making predictions
y_pred = bstI2P.predict(dtest)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)


<ipython-input-58-cd80b924d9a2>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_features['Label'] = label_encoder.fit_transform(df_relevant_features['Label'])


Accuracy: 0.9226993865030675
F1 Score: 0.9226692075187037


In [59]:
import xgboost as xgb

# Define the path where you want to save the model
model_path = 'xgboost_model.model'

# Train your XGBoost model (assuming 'bst' is your trained XGBoost model)
# Example:
# bst = xgb.train(params, dtrain, num_round)

# Save the trained model
bstI2P.save_model(model_path)

print("XGBoost model saved successfully.")


XGBoost model saved successfully.


In [60]:
import xgboost as xgb

# Define the path where the model is saved
model_path = 'xgboost_model.model'

# Load the saved XGBoost model
bstI2P = xgb.Booster()
bstI2P.load_model(model_path)

print("XGBoost model loaded successfully.")


XGBoost model loaded successfully.


In [61]:
import xgboost as xgb

def classify_traffic_xgb(df_features, model_path):
    """
    Classifies traffic using a trained XGBoost model.

    Parameters:
    - df_features (DataFrame): DataFrame containing features of the traffic data.
    - model_path (str): Path to the trained XGBoost model file.

    Returns:
    - predicted_classes (array-like): Predicted classes for the traffic data.
    """
    # Load the trained XGBoost model
    xgb_model = xgb.Booster()
    xgb_model.load_model(model_path)

    # Convert features to DMatrix format
    dmatrix = xgb.DMatrix(df_features)

    # Predict the traffic class
    predictions = xgb_model.predict(dmatrix)

    # Convert probability predictions to class labels
    predicted_classes = predictions.argmax(axis=1)

    return predicted_classes


In [62]:
import xgboost as xgb

# Define the path to your model and pcap file
model_path = '/content/xgboost_model.model'  # Make sure this is the correct path to your model file
pcap_file_path = '/content/drive/MyDrive/Colab Notebooks/pcap/i2p_00001_20200407201731.pcap'

# Load the trained XGBoost model
xgb_model = xgb.Booster()
xgb_model.load_model(model_path)

# Extract features from the pcap file
df_features = extract_features(pcap_file_path)

# Prepare the features for the model (make sure df_features is a DataFrame with the correct shape)
# The model expects a 2D array-like structure. If df_features is a Series, convert it to a DataFrame

# Check the predicted class and print the corresponding label
predictions = xgb_model.predict(xgb.DMatrix(df_features))
predicted_class = predictions.argmax()  # No need for axis=1 as predictions is a 1D array
print("Predicted Class:", predicted_class)


Predicted Class: 0


In [64]:
predicted_class = label_encoder.inverse_transform(predictionz.argmax(axis=1))[0]
print("Predicted Class:", predicted_class)


Predicted Class: chat


# **Flask**

In [17]:
!pip install flask
!pip install scapy

In [18]:
!pip install flask flask-ngrok


In [19]:
!pip install flask-ngrok

In [20]:
!pip install pyngrok

In [21]:
!ngrok authtoken '2cmVpIezM4pvk5PwnsBcrtZV5q3_6C4eAXASSwyqqrJH1qDqw'


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [26]:
pip install flask-cors


In [36]:
from pyngrok import ngrok

# Setup a tunnel to the Flask application on port 5000
ngrok_tunnel = ngrok.connect(5000)


In [35]:

from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
from flask_cors import CORS  # Ensure Flask-CORS is installed
import os
import numpy as np
import pandas as pd
from scapy.all import rdpcap, IP, TCP
import xgboost as xgb  # Ensure XGBoost is installed

app = Flask(__name__, template_folder='.')
app.config['UPLOAD_FOLDER'] = 'uploads'
# Automatically create the uploads directory if it doesn't exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
CORS(app)  # Enable CORS for all domains on all routes

# Load the XGBoost model (adjust the path to where your model is located)
MODEL_PATH = '/content/xgb_model.json'  # Update this path
xgb_model = xgb.XGBClassifier()
xgb_model.load_model(MODEL_PATH)

def preprocess_pcap(file_path):
    packets = rdpcap(file_path)
    features = {
        'Flow Duration': 0.0,
        'Total Fwd Packet': 0,
        'Total Bwd packets': 0,
        'Packet Length Min': np.inf,
        'Packet Length Mean': 0.0,
        'Fwd IAT Total': 0.0,
        'Flow IAT Min': np.inf,
        'Flow IAT Max': 0.0,
        'Fwd IAT Mean': 0.0,
        'Flow Packets/s': 0.0,
        'Flow Bytes/s': 0.0,
        'Idle Min': np.inf,
        'Idle Max': 0.0,
        'Idle Mean': 0.0,
        'Idle Std': 0.0,
        'FWD Init Win Bytes': 0,
        'Bwd Init Win Bytes': 0,
        'ACK Flag Count': 0
    }

    if not packets:
        return pd.DataFrame(features, index=[0])

    start_times = []
    packet_lengths = []
    iats = []
    total_bytes = 0

    for packet in packets:
        if IP in packet and TCP in packet:
            packet_length = len(packet)
            packet_lengths.append(packet_length)
            total_bytes += packet_length

            if 'S' in packet[TCP].flags:
                if features['FWD Init Win Bytes'] == 0:
                    features['FWD Init Win Bytes'] = packet[TCP].window
                else:
                    features['Bwd Init Win Bytes'] = packet[TCP].window

            if 'A' in packet[TCP].flags:
                features['ACK Flag Count'] += 1

            start_times.append(float(packet.time))

            if len(start_times) > 1:
                iat = start_times[-1] - start_times[-2]
                iats.append(iat)

    features['Flow Duration'] = max(start_times) - min(start_times)
    features['Total Fwd Packet'] = len([p for p in packets if IP in p and p[IP].src < p[IP].dst])
    features['Total Bwd packets'] = len([p for p in packets if IP in p and p[IP].src > p[IP].dst])
    features['Packet Length Min'] = min(packet_lengths)
    features['Packet Length Mean'] = np.mean(packet_lengths) if packet_lengths else 0
    features['Fwd IAT Total'] = sum(iats)
    features['Flow IAT Min'] = min(iats) if iats else 0
    features['Flow IAT Max'] = max(iats) if iats else 0
    features['Fwd IAT Mean'] = np.mean(iats) if iats else 0
    features['Flow Packets/s'] = len(packets) / features['Flow Duration'] if features['Flow Duration'] else 0
    features['Flow Bytes/s'] = total_bytes / features['Flow Duration'] if features['Flow Duration'] else 0

    # Convert to DataFrame
    df_features = pd.DataFrame([features])
    df_features.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_features.fillna(0, inplace=True)
    return df_features

# Prediction function using the preprocessed data
def predict_darknet(file_path):
    df_features = preprocess_pcap(file_path)
    # Assuming the model expects a DMatrix
    dmatrix = xgb.DMatrix(df_features)
    predictions = xgb_model.predict(dmatrix)
    return predictions

@app.route("/")
def home():
    return render_template("index.html")

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        predictions = predict_darknet(file_path)
        return jsonify({'prediction': str(predictions)})
    except Exception as e:
        # Log the exception to your Flask app log, if needed
        return jsonify({'error': 'Failed to process the file. Please try again.'}), 500

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
from flask_cors import CORS  # Ensure Flask-CORS is installed
import os
import numpy as np
import pandas as pd
from scapy.all import rdpcap, IP, TCP
import xgboost as xgb  # Ensure XGBoost is installed

app = Flask(__name__, template_folder='.')
app.config['UPLOAD_FOLDER'] = 'uploads'
# Automatically create the uploads directory if it doesn't exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
CORS(app)  # Enable CORS for all domains on all routes

# Load the XGBoost model (adjust the path to where your model is located)
MODEL_PATH = '/content/xgb_model.json'  # Update this path
xgb_model = xgb.XGBClassifier()
xgb_model.load_model(MODEL_PATH)

def preprocess_pcap(file_path):
    packets = rdpcap(file_path)
    features = {
        'Flow Duration': 0.0,
        'Total Fwd Packet': 0,
        'Total Bwd packets': 0,
        'Packet Length Min': np.inf,
        'Packet Length Mean': 0.0,
        'Fwd IAT Total': 0.0,
        'Flow IAT Min': np.inf,
        'Flow IAT Max': 0.0,
        'Fwd IAT Mean': 0.0,
        'Flow Packets/s': 0.0,
        'Flow Bytes/s': 0.0,
        'Idle Min': np.inf,
        'Idle Max': 0.0,
        'Idle Mean': 0.0,
        'Idle Std': 0.0,
        'FWD Init Win Bytes': 0,
        'Bwd Init Win Bytes': 0,
        'ACK Flag Count': 0
    }

    if not packets:
        return pd.DataFrame(features, index=[0])

    start_times = []
    packet_lengths = []
    iats = []
    total_bytes = 0

    for packet in packets:
        if IP in packet and TCP in packet:
            packet_length = len(packet)
            packet_lengths.append(packet_length)
            total_bytes += packet_length

            if 'S' in packet[TCP].flags:
                if features['FWD Init Win Bytes'] == 0:
                    features['FWD Init Win Bytes'] = packet[TCP].window
                else:
                    features['Bwd Init Win Bytes'] = packet[TCP].window

            if 'A' in packet[TCP].flags:
                features['ACK Flag Count'] += 1

            start_times.append(float(packet.time))

            if len(start_times) > 1:
                iat = start_times[-1] - start_times[-2]
                iats.append(iat)

    features['Flow Duration'] = max(start_times) - min(start_times)
    features['Total Fwd Packet'] = len([p for p in packets if IP in p and p[IP].src < p[IP].dst])
    features['Total Bwd packets'] = len([p for p in packets if IP in p and p[IP].src > p[IP].dst])
    features['Packet Length Min'] = min(packet_lengths)
    features['Packet Length Mean'] = np.mean(packet_lengths) if packet_lengths else 0
    features['Fwd IAT Total'] = sum(iats)
    features['Flow IAT Min'] = min(iats) if iats else 0
    features['Flow IAT Max'] = max(iats) if iats else 0
    features['Fwd IAT Mean'] = np.mean(iats) if iats else 0
    features['Flow Packets/s'] = len(packets) / features['Flow Duration'] if features['Flow Duration'] else 0
    features['Flow Bytes/s'] = total_bytes / features['Flow Duration'] if features['Flow Duration'] else 0

    # Convert to DataFrame
    df_features = pd.DataFrame([features])
    df_features.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_features.fillna(0, inplace=True)
    return df_features

# Prediction function using the preprocessed data
def predict_darknet(file_path):
    df_features = preprocess_pcap(file_path)
    # Assuming the model expects a DMatrix
    dmatrix = xgb.DMatrix(df_features)
    predictions = xgb_model.predict(dmatrix)
    return predictions

@app.route("/")
def home():
    return render_template("index.html")

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        predictions = predict_darknet(file_path)
        return jsonify({'prediction': str(predictions)})
    except Exception as e:
        # Log the exception to your Flask app log, if needed
        return jsonify({'error': 'Failed to process the file. Please try again.'}), 500

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
